## Evaluation of Models on Incident Prediction

In [2]:
import numpy as np
import torch

from torchmetrics import Accuracy
from torchmetrics.functional import precision_recall
from tqdm import tqdm

from train import create_parser
from models import *

/home/ubuntu/anaconda3/envs/traffic/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Load Model and Data

In [3]:
parser = create_parser()
args = parser.parse_args(args=[])

args.use_density = True
args.use_truck_spd = True
args.use_pv_spd = True
args.use_speed = True
args.use_expectation = True
args.device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [4]:
traffic_model = TrafficModel(args)
traffic_model_path = "./checkpoints/finetune/best_exp_tmc_T_T_T_7_6_5_T.pt"
base_model = TrafficSeq2Seq(args)
base_model_path = "./checkpoints/base/best_exp_tmc_T_T_T_7_6_5_T.pt"

with open(base_model_path, "rb") as f_base_model, open(traffic_model_path, "rb") as f_traffic_model:
        # load state dict
        state_dict_base = torch.load(f_base_model, map_location=args.device)
        state_dict_traffic = torch.load(f_traffic_model, map_location=args.device)

        # populate state dict into models and log
        base_model.load_state_dict(state_dict_base)
        traffic_model.load_state_dict(state_dict_traffic)

In [5]:
X_path = "./data/new_X.npy"
Y_path = "./data/new_Y_tmc.npy"

input = torch.from_numpy(np.load(X_path)).float()  # (29520, 1796)
output = torch.from_numpy(np.load(Y_path)).float()   # (29520, 70, 4)


In [6]:
days = 164
daily_slots = 180

X = []  # serve as model input
Y = []  # serve as model input
for d in range(days):
    for i in range(daily_slots-args.in_seq_len+1):
        X.append(input[d*days+i : d*days+i+args.in_seq_len, :])
        Y.append(output[d*days+i : d*days+i+args.in_seq_len, ...])
X = torch.stack(X, dim=0)  # (28536, 7, 1796)
Y = torch.stack(Y, dim=0)  # (28536, 7, 70, 4)

target = []
for d in range(days):
    for i in range(1, daily_slots-args.out_seq_len+1):
        target.append(output[d*days+i : d*days+i+args.out_seq_len, ...])
target = torch.stack(target, dim=0)  # (28536, 6, 70, 4)
inc_target = target[...,3].int()  # (28536, 6, 70)
spd_target = target[..., :3]  # (28536, 6, 70, 3)


In [7]:
with torch.no_grad():
    traffic_model.eval()
    base_model.eval()
    temp_X = X[:24]
    temp_Y = Y[:24]
    traffic_spd_pred, traffic_inc_pred, _ = traffic_model(temp_X, temp_Y)
    base_spd_pred, _, _ = base_model(temp_X, temp_Y)

In [8]:
batch_size = 24
traffic_spd_pred, traffic_inc_pred, base_spd_pred = [], [], []
with torch.no_grad():
    traffic_model.eval()
    base_model.eval()
    for i in tqdm(range(1189)):
        temp_X = X[(i*24) : (i+1)*24]
        temp_Y = Y[(i*24) : (i+1)*24]
        temp_traffic_spd_pred, temp_traffic_inc_pred, _ = traffic_model(temp_X, temp_Y)
        temp_base_spd_pred, _, _ = base_model(temp_X, temp_Y)

        traffic_spd_pred.append(temp_traffic_spd_pred)
        traffic_inc_pred.append(temp_traffic_inc_pred)
        base_spd_pred.append(temp_base_spd_pred)

traffic_spd_pred = torch.cat(traffic_spd_pred, dim=0)  # (28536, 6, 210)
traffic_inc_pred = torch.cat(traffic_inc_pred, dim=0)  # (28536, 6, 70)
base_spd_pred = torch.cat(base_spd_pred, dim=0)  # (28536, 6, 210)

100%|██████████| 1189/1189 [00:46<00:00, 25.55it/s]


In [1]:
np.save("traffic_spd_pred_tmc.npy", traffic_spd_pred.cpu().detach().numpy())
np.save("traffic_inc_pred_tmc.npy", sigm(traffic_inc_pred).cpu().detach().numpy())
np.save("base_spd_pred_tmc.npy", base_spd_pred.cpu().detach().numpy())

NameError: name 'np' is not defined

In [41]:
sigm = torch.nn.Sigmoid()
thresholds = [0.05*i for i in range(20)]

for t in thresholds:
    accu_metric = Accuracy(t)
    print(accu_metric(sigm(traffic_inc_pred), inc_target))
    print(precision_recall(sigm(traffic_inc_pred), inc_target, threshold=t))

tensor(0.0265)
(tensor(0.0265), tensor(1.))
tensor(0.0265)
(tensor(0.0265), tensor(1.))
tensor(0.0265)
(tensor(0.0265), tensor(1.))
tensor(0.0265)
(tensor(0.0265), tensor(1.))
tensor(0.0265)
(tensor(0.0265), tensor(1.))
tensor(0.0265)
(tensor(0.0265), tensor(1.0000))
tensor(0.0265)
(tensor(0.0265), tensor(1.0000))
tensor(0.0266)
(tensor(0.0265), tensor(0.9999))
tensor(0.0269)
(tensor(0.0265), tensor(0.9997))
tensor(0.0275)
(tensor(0.0265), tensor(0.9991))
tensor(0.0294)
(tensor(0.0265), tensor(0.9976))
tensor(0.0351)
(tensor(0.0265), tensor(0.9935))
tensor(0.0523)
(tensor(0.0267), tensor(0.9808))
tensor(0.0983)
(tensor(0.0270), tensor(0.9456))
tensor(0.1992)
(tensor(0.0276), tensor(0.8554))
tensor(0.4176)
(tensor(0.0288), tensor(0.6434))
tensor(0.7402)
(tensor(0.0320), tensor(0.3014))
tensor(0.9278)
(tensor(0.0394), tensor(0.0739))
tensor(0.9706)
(tensor(0.0472), tensor(0.0058))
tensor(0.9735)
(tensor(0.3220), tensor(5.9920e-05))


In [1]:
train_inc_pred

NameError: name 'train_inc_pred' is not defined